## Code to prepare top ranked reviews for each question

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import spatial

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
reviews = pd.read_csv("/kaggle/input/ir-project-bert/Reviews_126Products.csv")
ques_ans = pd.read_csv("/kaggle/input/q-a-new/Questions_Answers_126Products.csv")

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
reviews.head()

In [ ]:
ques_ans.tail(10)

In [ ]:
reviews.shape, ques_ans.shape

In [ ]:
reviews["Review_Text"] = reviews.Review_Text.str.lstrip("'")

In [ ]:
#### Extracting best answer out of all the answers of that question

In [ ]:
new_ques_ans = pd.DataFrame(columns = ["Prod_ID", "Ques_ID", "Ans_ID", "Q_A"])

quesid = 0
old = 0
for ans in range(0, ques_ans.shape[0]):
    
    if (ques_ans["Ques_ID"][ans] == quesid):
        
        alist = []
        alist.append(ques_ans["Question Text"][ans])
        alist.append(ques_ans["Ans_Text"][ans])
        
        sentence_embeddings = model.encode(alist)
    
        dataSetI = sentence_embeddings[0]
        dataSetII = sentence_embeddings[1]
        
        result = 1 - spatial.distance.cosine(dataSetI, dataSetII)
        print(result)
        
        if (result>old):
            print("got new")
            old  = result
            answerid = ques_ans["Ans_ID"][ans]
            idx_final = ans
            
        if((ans < ques_ans.shape[0]-1) & (ques_ans["Ques_ID"][ans+1] != quesid)):
            print("appending in new df")
            new_ques_ans = new_ques_ans.append({ "Prod_ID": ques_ans["Prod_ID"][idx_final],
                                                 "Ques_ID": ques_ans["Ques_ID"][idx_final],
                                                 "Ans_ID": ques_ans["Ans_ID"][idx_final],
                                                 "Q_A": ques_ans["Question Text"][idx_final] + ques_ans["Ans_Text"][idx_final]}, ignore_index = True)
            
            old=0
            quesid = quesid+1
            
        elif(ans == ques_ans.shape[0]-1):
            print("appending in new df")
            new_ques_ans = new_ques_ans.append({ "Prod_ID": ques_ans["Prod_ID"][idx_final],
                                                 "Ques_ID": ques_ans["Ques_ID"][idx_final],
                                                 "Ans_ID": ques_ans["Ans_ID"][idx_final],
                                                 "Q_A": ques_ans["Question Text"][idx_final] + ques_ans["Ans_Text"][idx_final]}, ignore_index = True)
            
    

In [ ]:
new_ques_ans["Q_A"][0]

In [ ]:
new_ques_ans.head()

In [ ]:
new_ques_ans.tail()

In [ ]:
new_ques_ans.shape

In [ ]:
##### Extracting top 5 reviews pertaining to the best question-answer pair for each question

In [ ]:
scores = pd.DataFrame(columns = ["Prod_ID", "Ques_ID", "Ans_ID", "Review_ID", "score"])

for qa in range(0, new_ques_ans.shape[0]):
        
    pdtid = new_ques_ans["Prod_ID"][qa]
    review_temp = reviews[reviews["Product ID"]==pdtid].reset_index()
    
    print(pdtid)
    print(qa)
    
    for r in range(0, review_temp.shape[0]):
        
        llist = []
        llist.append(new_ques_ans["Q_A"][qa])
        llist.append(review_temp["Review_Text"][r])
        
        sentence_embeddings = model.encode(llist)
        
        dataSetI = sentence_embeddings[0]
        dataSetII = sentence_embeddings[1]
        
        result = 1 - spatial.distance.cosine(dataSetI, dataSetII)
        
        scores = scores.append({ "Prod_ID": int(new_ques_ans["Prod_ID"][qa]),
                                             "Ques_ID": int(new_ques_ans["Ques_ID"][qa]),
                                             "Ans_ID": int(new_ques_ans["Ans_ID"][qa]),
                                             "Review_ID": int(review_temp["Review_ID"][r]),
                                             "score": result}, ignore_index = True)

In [ ]:
scores.head()

In [ ]:
int(new_ques_ans["Prod_ID"][0])

In [ ]:
scores.to_csv("ranking_scores.csv")
from IPython.display import FileLink
FileLink(r'ranking_scores.csv')

In [ ]:
scores.head(20)

In [ ]:
scores.shape

In [19]:
df1 = scores.groupby(["Ques_ID"])
df2 = df1.apply(lambda x: x.sort_values(["score"], ascending = False))
df3 = df2.reset_index(drop=True)

In [21]:
df3.head(20)

,Unnamed: 0,Prod_ID,Ques_ID,Ans_ID,Review_ID,score
0,10,1.0,0.0,2.0,10.0,0.891508
1,23,1.0,0.0,2.0,23.0,0.881397
2,11,1.0,0.0,2.0,11.0,0.865133
3,53,1.0,0.0,2.0,53.0,0.857536
4,27,1.0,0.0,2.0,27.0,0.849587
5,46,1.0,0.0,2.0,46.0,0.848951
6,40,1.0,0.0,2.0,40.0,0.845542
7,36,1.0,0.0,2.0,36.0,0.843963
8,47,1.0,0.0,2.0,47.0,0.842872
9,22,1.0,0.0,2.0,22.0,0.840668


In [22]:
mapping = df3.groupby('Ques_ID').head(5)

In [23]:
mapping.head(20)

,Unnamed: 0,Prod_ID,Ques_ID,Ans_ID,Review_ID,score
0,10,1.0,0.0,2.0,10.0,0.891508
1,23,1.0,0.0,2.0,23.0,0.881397
2,11,1.0,0.0,2.0,11.0,0.865133
3,53,1.0,0.0,2.0,53.0,0.857536
4,27,1.0,0.0,2.0,27.0,0.849587
71,117,1.0,1.0,3.0,46.0,0.852403
72,82,1.0,1.0,3.0,11.0,0.821392
73,95,1.0,1.0,3.0,24.0,0.800117
74,81,1.0,1.0,3.0,10.0,0.798409
75,135,1.0,1.0,3.0,64.0,0.793919


In [ ]:
mapping.to_csv("mapping.csv")
from IPython.display import FileLink
FileLink(r'mapping.csv')